# <center> Project 01: Machine Learning for customer segmentation through clustering</center>

# Seminar goals:
In this project, we will develop and apply different clustering tecnhiques. Unsupervised methodologies are broadly used in business to generate **insights** from several type of data sources as text or image and they are a baseline tool for multiple areas marketing, sales or customer care. 

In our case, we will use clustering techniques to understand a customer base; i.e. describe the main characteristics of our customers for **marketing purposes**.

During this project we will follow the end-to-end Machine Learning process: from data gathering and cleaning, exploratory data analysis, feature engineering and finally, training and prediction.
The project is structured as follows:

![<framework_ml_project>](framework_ml_project.png)


In this project, we will apply two clustering techniques: K-means and Mixture of Gaussians. Both techniques follow different approach to cluster or segment the data. We will learn how to apply them using Python and how they are used to generate insights about our customer base, i.e. identify the main types or **sterotypes** of customers and their differences. Besides, we will learn to calculate the optimal K value and measure the quality of the clustering.

### Due date: up to May, 20th, 23:55h. 
### Submission procedure: via Moodle.

*******

# Step 0. Understanding the problem: customers' stereotypes 

Recently, we have joined a data scientist and AI engineering team of a telco company. This team is supporting decision making process of several internal areas as marketing and customer care.

Our marketing colleagues are planning to launch a new commercial campaign for a new mobile tariff. As input for the tariff design, the Marketing product managers needs to know the **pattern** or **stereotypes** of our current customers: i.e. the main customers' sectors (Industry, Services, Agriculture, ...), number of employees, Turnover, which products they consume, how much they spend in telco services, etc. 

To support marketing demands, we are going **to use two of the most important clustering techniques: K-means and Mixture of Gaussians (MoG).**


# Step 1: Data gathering


![<data_gathering>](data_gathering.png)


In this practice we are using a new dataset named `customer_dt.csv`. This file contains information of **272 of our telco's consumers** for 3 months. In particular, the detailed information for each customer is:

- *Company_id*: It's an integer that identifies any company. Each value is unique for every company.
- *Reference_date_month*: It refers to the month corresponding to the customer information
- *Product*: Name level 1 of the product
- *Sub_product*: Name level 2 of the product
- *Type_ID_Coverage_GSM*: type of coverage for GSM (2G) network: indoor, outdoor or no-coverage
- *Type_ID_Coverage_UMTS*: type of coverage for UMTS (3G) network: indoor, outdoor or no-coverage
- *Type_ID_Coverage_LTE*: type of coverage for LTE (4G) network: indoor, outdoor or no-coverage
- *CNT_EMPLOYEE: Number of employees of the company
- *Sector*: It's an integer that identifies the sector of the company's activity
- *Sub_sector*: It's an integer that identifies the sub_sector of the company's activity
- *Turnover*: The annual incomes of the company
- *ZIP*: Postal code where the company is placed
- *Data_usage*: Number of Gigabits for this product and sub_product and Reference_date_month 
- *Voice_usage*: Number of minutes for this product and sub_product and Reference_date_month 
- *Monthly_expense*: Euros expended in this product and sub_product and Reference_date_month
- *N_lines*: Number of mobile lines this product and sub_product and Reference_date_month

# Step 2: Data understanding and preparation

Once we know the problem to solve, the next stage is to have a clear understanding of the data we have extracted and to prepare it before modelling. In particular, we will:
- List and verify the type of each variable (object, float, int...). Identify variables with nulls. Measure the memory usage
- Eliminate rows with nulls in order to have a dataset 100% fulfilled
- Aggregate rows with monthly expense per customers in order to have just 1 sample per customers
- Exploratory Data Analysis to understand main statistics (mean, standard deviation, min&max values and 25%-50%-75% quartiles) and distribution of the most relevant variables or features as data usage, voice usage, monthly expense and number of lines
- Plot several graphs in order to identify how variables are related between them. In particular:
- correlation matrix
- 2D and 3D scatter plots between data usage, voice usage and monthly expense

Once this part, also known as **data wrangling** of the Project is done, we should achieve a deep knowledge about the data. Besides, the dataset will have been processed to be ready to apply the clustering algorithms to solve the business problem.

Let's import the main Python libraries required in our project.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import pyplot
from mpl_toolkits import mplot3d
from scipy.stats import chi2_contingency
from sklearn.metrics import pairwise_distances_argmin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse

Open the csv with separator ";" and assign to a dataframe variable (use read_csv from Pandas library). Let's see the top 5 elements.

In [ ]:
customer_dt = pd.read_csv('customer_dt.csv', sep=";")
customer_dt.head()

[**EX1**] [**REPORT**] Let's identify the type of the variables (integer, float, chart...) and the size of the dataset and the file. Which is the variable with more nulls? And with no nulls? 

Tip: [.info()](https://www.geeksforgeeks.org/python-pandas-dataframe-info/) is a function that reports the main characteristics of a dataframe.

In [ ]:
customer_dt.info()

In our dataset there a subset of variables that describe a company regardless of the month, i.e. `Company_id`, `CNT_EMPLOYEE`, `Sector`, `Sub_sector`, `Turnover`, `Revenue_Scale` and `ZIP`. We should guarantee that our dataset for training the cluster has no **nulls** in those variables.  

[**EX2**] [**CODE**] Eliminate those rows with nulls in any of the descriptive company variables, i.e. `Company_id`, `CNT_EMPLOYEE`, `Sector`, `Sub_sector`, `Turnover`, `Revenue_Scale` and `ZIP`.

Tip: you may have to use the [set_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html) function.

In [ ]:
#Usaremos el codigo del tutorial como referencia
print("Removing nulls\n")
customer_dt=customer_dt[(customer_dt['Company_id'].isna()==False) & (customer_dt['CNT_EMPLOYEE'].isna()==False) 
                        & (customer_dt['Sector'].isna()==False) & (customer_dt['Sub_sector'].isna()==False)& (customer_dt['Turnover'].isna()==False)& (customer_dt['Revenue_Scale'].isna()==False)& (customer_dt['ZIP'].isna()==False)]
print("The resulted lenght is:", len(customer_dt))

Let's re-calculate the type of the variables (integer, float, chart...) and the size of the dataset and the file. Your output should look like this:

In [ ]:
customer_dt.info()

As we observe, there are still few rows with **null** values in `TYPE_ID_Coverage_GSM`, `TYPE_ID_Coverage_UMTS` and `TYPE_ID_Coverage_LTE`. Let's understand which registers are.

[**EX3**] [**CODE**] Create a `non_coverage_register` variable with the rows with **nulls** in these coverage variables. How many registers does the new `non_coverage_register` variable have?

In [ ]:
#De forma muy parecida al ejercicio 2, usamos como referencia lo visto en el tutorial
non_coverage_register = customer_dt[(customer_dt['TYPE_ID_Coverage_GSM'].isna()==True)&(customer_dt['TYPE_ID_Coverage_UMTS'].isna()==True)&(customer_dt['TYPE_ID_Coverage_LTE'].isna()==True)]

print("The resulted lenght is:", len(non_coverage_register))
non_coverage_register

As **nulls** in coverage only affects one customer, we can decide to remove it from the training dataset or to impute an statistic value as the mean, median or most frequent value in each of these nulls.

[**EX4**] [**CODE**] Remove the customers with **nulls** in coverage variables.

In [ ]:
#Lo mismo que en el ejercicio dos pero para estas variables que aun tienen nulos
customer_dt=customer_dt[(customer_dt['TYPE_ID_Coverage_GSM'].isna()==False) & (customer_dt['TYPE_ID_Coverage_UMTS'].isna()==False) 
                        & (customer_dt['TYPE_ID_Coverage_LTE'].isna()==False)]

customer_dt.info()

As explained before, our original dataset includes **up to 3 months of telco consumption (voice, data and number of mobile lines) and expense** per customer.

[**EX5**] [**CODE**] Create a new dataset named `customer_dt_summary` with only one register per `Company_id` and `Sub_product` category with the average value in `Data_usage`, `Voice_usage` and `Monthly_expense` and `N_lines`. Which is the size of your new dataset? Verify this new dataset does not contain **nulls**.

In [ ]:
#Calcula la media de Data_usage, Voice_usage, Monthly_expense y N_lines para un solo registro por Company_id y Sub_product

#customer_dt_summary = customer_dt[['Company_id','Sub_product','Data_usage', 'Voice_usage', 'Monthly_expense', 'N_lines']]
#customer_dt_summary = customer_dt_summary.groupby(["Company_id", "Sub_product"]).mean()
customer_dt_summary = customer_dt.groupby(["Company_id", "Sub_product"])[['Data_usage', 'Voice_usage', 'Monthly_expense', 'N_lines']].mean()

print(customer_dt_summary)

In [ ]:
print("The resulted lenght is:", len(customer_dt_summary))

In Machine Learning, it is key to understand the nature of the data before training. For numeric variables, it is useful to calculate the distribution and main statistics.

[**EX6**] [**REPORT**] Calculate the main statistics (max, min, mean, median and standard deviation) of `Data_usage`, `Voice_usage`, `Monthly_expense` and `N_lines` variables. Plot a histogram for each of these variables

Tip: use [Seaborn library](https://seaborn.pydata.org/) with `kde=False` to create a histogram. You also can use **dataframe_column.hist(bins=number_of_bins)**

In [ ]:
print("*********** MAX ***********")
maxs = customer_dt[['Data_usage','Voice_usage','Monthly_expense','N_lines']].max()
print(maxs)

print("*********** MIN ***********")
mins = customer_dt[['Data_usage','Voice_usage','Monthly_expense','N_lines']].min()
print(mins)

print("*********** MEAN ***********")
mean = customer_dt[['Data_usage','Voice_usage','Monthly_expense','N_lines']].mean()
print(mean)

print("*********** MEDIAN ***********")
median = customer_dt[['Data_usage','Voice_usage','Monthly_expense','N_lines']].median()
print(median)

print("*********** STD ***********")
std = customer_dt[['Data_usage','Voice_usage','Monthly_expense','N_lines']].std(ddof=0)
print(std)

In [ ]:
#Histogrma tipo matlab
customer_dt['Data_usage'].hist(bins=6,color='yellow',ec="black",figsize=(3, 3))
plt.show()

customer_dt['Voice_usage'].hist(bins=6,color='yellow',ec="black",figsize=(3, 3))
plt.show()

customer_dt['Monthly_expense'].hist(bins=6,color='yellow',ec="black",figsize=(3,3))
plt.show()

customer_dt['N_lines'].hist(bins=6,color='yellow',ec="black",figsize=(3,3))
plt.show()

Additionaly to understanding each individual variable, it is important to understand how they are related to each other. 

[**EX7**] [**REPORT**] Calculate and plot the correlation matrix between traffic attributes (i.e. `Voice_usage` and `Data_usage`), `Monthly_expense`, `N_lines`, `Turnover` and `CNT_EMPLOYEE`. 
- Which are the variables with more and less correlation with respect to the `Monthly_expense` variable?
- Which are the top 2 variables with higher correlation with `Voice_usage`?
- Is `Data_usage` correlated with `Turnover`? Does it mean that a company spend more in `Data usage` when its `Turnover`increases? 
- Which is the highest correlated variable with `CNT_EMPLOYEE`?
- A company with high `Voice traffic` consumption but low `Data traffic`uses to spend more that other company with high `Data traffic`and low `Voice traffic`? Justify your answer.

Tip: use [pandas.DataFrame.corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html) to compute a correlation matrix, and [matplotlib.pyplot.matshow](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.matshow.html) to show this graphically.

In [ ]:
#Visualizamos primero los valores de correlacion
customer_dt_atr=customer_dt[['Voice_usage','Data_usage','Monthly_expense', 'N_lines', 'Turnover', 'CNT_EMPLOYEE']]
f = plt.figure(figsize=(19, 15))

#Para la matriz de correlacion usamos lo que nos dice el tip
plt.matshow(customer_dt_atr.corr(), fignum=f.number)
plt.xticks(range(customer_dt_atr.shape[1]), customer_dt_atr.columns, fontsize=14, rotation=45)
plt.yticks(range(customer_dt_atr.shape[1]), customer_dt_atr.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
customer_dt_atr.corr()

Another option to analyze the relation 1-to-1 between 2 variables in through scatter plots. Let's simplify the original dataset and create a new `training_dt`dataset with only `Voice_usage`, `Data_usage` and `Monthly_expense`.

In [ ]:
#Usariamos la primera linea que hemos usado antes, pero para menos variables
training_dt=customer_dt_summary[['Voice_usage', 'Data_usage', 'Monthly_expense']]
training_dt.head()

[**EX8**] [**REPORT**] Visualize a scatter plot with `Voice_usage` vs `Monthly_expense` variables. Could you visually identify any cluster? How many? Repeat the plot with registers which `Voice_usage`is between 0 and 10000 minutes. Could you identify any cluster?

In [ ]:
x = customer_dt.Voice_usage
y = customer_dt.Monthly_expense

plt.scatter(x, y)
plt.xlabel('Voice_usage')
plt.ylabel('Monthly_expense')
plt.show()
plt.scatter(x,y)

#Para hacer el zoom
plt.ylim(0,3000)
plt.xlim(0,10000)

plt.xlabel('Voice_usage')
plt.ylabel('Monthly_expense')
plt.show()

[**EX9**] [**REPORT**] Visualize a scatter plot with `Data_usage` vs `Monthly_expense` variables. Could you visually identify any cluster? How many?Repeat the plot with registers which `Data_usage`is between 0 and 500 GB. Could you identify any cluster?

In [ ]:
x = customer_dt.Data_usage
y = customer_dt.Monthly_expense

plt.scatter(x, y)
plt.xlabel('Data_usage')
plt.ylabel('Monthly_expense')
plt.show()
plt.scatter(x,y)

plt.ylim(0,5000)
plt.xlim(0,500)

plt.xlabel('Data_usage')
plt.ylabel('Monthly_expense')
plt.show()

[**EX10**] [**REPORT**] To improve our understanding of the data, plot a 3D visualization between `Voice_usage`, `Data_usage`and `Monthly_expense` for a new subset of the dataset where `Voice_usage`is below 10000 minutes and `Data_usage`is <=100 GB.
- Could you visually identify any cluster? How many?
- Could you identify any **outlier**?
- Could you identify a cluster bigger than the others? Describe approximately it in terms of the values of these 3 variables


Tip: use [scatter3d](https://matplotlib.org/3.1.1/gallery/mplot3d/scatter3d.html) to create 3D scatter plots.

If we rotate the 3D scatter plot, the output looks like as follows:

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
training_dt=customer_dt_summary[['Voice_usage', 'Data_usage', 'Monthly_expense']]
#Hacemos un zoom de esta otra manera
training_dt_filtered=training_dt[(training_dt['Voice_usage']<=10000) & (training_dt['Data_usage']<=100)]

# Data for a three-dimensional line
X=training_dt_filtered.loc[:,'Voice_usage'].values
Y=training_dt_filtered.loc[:,'Data_usage'].values
Z=training_dt_filtered.loc[:,'Monthly_expense'].values

ax.set_xlabel('Voice_usage')
ax.set_ylabel('Data_usage')
ax.set_zlabel('Monthly_expense')
ax.scatter3D(X, Y, Z, c=Z, cmap='viridis', linewidth=5);
#Angulo para visualizar
ax.view_init(0, -60)
plt.show()

If we rotate the 3D scatter plot, the output looks like as follows:

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
training_dt=customer_dt_summary[['Voice_usage', 'Data_usage', 'Monthly_expense']]
#Hacemos un zoom de esta otra manera
training_dt_filtered=training_dt[(training_dt['Voice_usage']<=10000) & (training_dt['Data_usage']<=100)]

# Data for a three-dimensional line
X=training_dt_filtered.loc[:,'Voice_usage'].values
Y=training_dt_filtered.loc[:,'Data_usage'].values
Z=training_dt_filtered.loc[:,'Monthly_expense'].values

ax.set_xlabel('Voice_usage')
ax.set_ylabel('Data_usage')
ax.set_zlabel('Monthly_expense')
ax.scatter3D(X, Y, Z, c=Z, cmap='viridis', linewidth=5);
#Angulo para visualizar
ax.view_init(90, -60)
plt.show()

# Step 3: Training the model and performance evaluation: Segmentation of customers through K-means clustering

Once the dataset has been processed and we have a first understanding of the type and characteristics of the variables, we are ready to apply clustering methods to group the register to answer the key marketing question: How are our customers?

Firstly, we will code our own Kmeans algorithm. As the dataset has a high number of features, we will select voice usage, data usage and monthly expense variables to fit the clusters.
Once the clustering is done, we need to understand the output. 2-dimension and 3-dimension scatter plot visualizations are excellent techniques to evaluate the clustering output.
To check if our Kmeans algorithm works properly, we will use the Sklearn’s Kmeans function to cluster the dataset. We will compare the 2D and 3D plots from the Sklearn clustering and ours.
Finally, as part of any Machine Learning Project, we need to calculate the perfomance of our model. For Kmeans, we will 1) estimate the optimal K value through the Elbow method and 2) calculate the sihouette score for several values of K


## Your own K-means function

[**EX11**] [**CODE**] Build a `calculate_distance` function to calculate the distance between each point and the centroid

In [ ]:
#Debido al enunciado no nos quedo muy claro si debiamso calcular la distancia para un solo punto, un solo centroide
#o todos los puntos juntos, asi que para que fuera mas limpia la formula siguiente, consideramos que
#recibimos todo el dataset como X y que centroid es todo los centroides, asi ponemos todo en conjunto
def calculate_distance (X, centroid):
    squareDistance=np.zeros((X.shape[0],centroid.shape[0]))
    #Calculamos las distancias de cada punto del dataset a cada uno de los centroides con dos for
    for i in range(len(X)): #Este for nos recorrera todo el dataset X
        x_val = np.array(X[i,:])
        for j in range(len(centroid)): #Este recorrera todos los centroides
            #distancia al cuatrado seria
            centroid_val = np.array(centroid[j,:])
            vect = x_val - centroid_val
            #la diferencia entre x y el centroide
            squareDistance[i,j] = np.sqrt(np.dot(vect.T, vect)) #y luego la raiz de los cuadrados para obtener la distancia
    return squareDistance

[**EX12**] [**CODE**] Build `K_means_clustering` function that creates a clustering according to K-means methodology.

In [ ]:
from sklearn.metrics import pairwise_distances_argmin
def K_means_clustering(X, n_clusters=2, seed=1, num_iterations=10):
    # Initialitize centroids based on a random selection of #n_clusters samples of X 
    rng = np.random.RandomState(seed)
    i = rng.permutation(X.shape[0])[:n_clusters] 
    #esta instruccion nos dio problemas y tuvimos que acceder a los valores de X con iloc, pero al parecer
    #era un problema que surgia si no pasabamos el dataframe correctamente
    centroids = X[i] 
    
    #Repeat the process during num_iterations or convergence achieved
    for num in range(0,num_iterations):
    
    #For each iteration, calculate the shortest distance of each point of X to centroids
    #Labels are based on the index in the centroids array

        #Tambien es posible usar la función pairwise_distances_argmin para calcular los indices 
        #de los centroides tal que la distancia a cada punto es minima, de hecho es una instruccion que viene
        #importada por defecto
        #labels = pairwise_distances_argmin(X, centroids)
        
        #También, y la opcion que tiene mas sentido, podemos usar la función que hemos creado anteriormente
        distances_to_centroids = calculate_distance(X, centroids)
        labels = np.argmin(distances_to_centroids, axis=1)
        
    #Calculate the new centroids based on the means of each point assigned to each cluster 
        
        #Es aplicar el calculo de la media solo a las X que pertenerzcan a cierto label (numero de cluster)
        new_centroids = np.array([X[labels == i].mean(0) for i in range(n_clusters)]) #asi tenemos los nuevos centroides
           
    # Evaluate convergence: if new_centroids=centroids, stop iterations
    
        if np.all(centroids == new_centroids):
            #Se converge
            print('Convergence achieved with:',num, 'iterations')
            break #hacmeos break para detener el programa
        else:
            if num%10 == 0 and num != 0:
                #no se converge
                print('No convergence yet after', num, 'iterations')
        centroids = new_centroids
        
    return centroids, labels

Let's define the `training_dt` dataset based on the following variables: `Voice_usage`, `Data_usage`, `Monthly_expense`.

In [ ]:
training_dt=customer_dt_summary[['Voice_usage', 'Data_usage', 'Monthly_expense']]

[**EX13**][**CODE**] Execute your `K_means_clustering` function to `training_dt` and number of clusters=3. Calculate the centroids of each cluster. How many iterations are necessary to achieve the clustering?

In [ ]:
#Importante usar training_dt.values y no training_dt para acceder a los valores del dataframe
centroids, labels = K_means_clustering(training_dt.values, n_clusters=3, seed=1, num_iterations=100)

print('Centroides de cada cluster:')
centroids

Now, it's time to understand how the clustering process works! To do it, we are plotting the `training_dt` painting the colour based on `Cluster_id`, output from the k-means. 

 [**EX14**][**REPORT**] Plot the following scatter plots representing the centroids:
 - Between `Data_usage` vs `Voice_usage` 
 - Between `Monthly_expense`vs `Voice_usage` and
 - Between `Monthly_expense`vs `Data_usage`
 
 Describe the minority cluster in terms of `Data_usage`, `Voice_usage`and `Monthly_expense`. How many register is if formed by?

In [ ]:
voice = training_dt.loc[:,'Voice_usage']
data = training_dt.loc[:,'Data_usage']
month_expense = training_dt.loc[:,'Monthly_expense']

#Data_usage vs Voice_usage
fig = plt.figure()
plt.scatter(voice,data,c=labels,cmap='viridis')
plt.xlabel('Voice_usage')
plt.ylabel('Data_usage')
plt.show()

#Voice_usage vs Monthly_expense
fig = plt.figure()
plt.scatter(voice,month_expense,c=labels,cmap='viridis')
plt.xlabel('Voice_usage')
plt.ylabel('Monthly_expense')
plt.show()

#Data_usage vs Monthly_expense
fig = plt.figure()
plt.scatter(data,month_expense,c=labels,cmap='viridis')
plt.xlabel('Data_usage')
plt.ylabel('Monthly_expense')
plt.show()

[**EX15**] [**CODE**] [**REPORT**] Execute the Sklearn library's KMeans function and compare both `Data_usage`vs `Voice_usage`scatter plots. Are they similar?

Tip: We recommend the following  [KMeans()](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) parameters: `init`='random', `n_init`=10, `tol`=1e-04 and `random_state`=0

In [ ]:
#Es como el ejercicio anterior, pero usamos el modelo KMeans de la libreria que hay importada por defecto
kmeans = KMeans(n_clusters=3, init='random',n_init=10,tol=1e-04,random_state=0).fit(training_dt)
labels = kmeans.predict(training_dt)

voice = training_dt.loc[:,'Voice_usage']
data = training_dt.loc[:,'Data_usage']

fig = plt.figure()
plt.scatter(voice,data,c=labels)
plt.xlabel('Voice_usage')
plt.ylabel('Data_usage')
plt.show()

[**EX16**] [**REPORT**] Repeat the 3D plot visualization between `Voice_usage`, `Data_usage`and `Monthly_expense` after the clustering process. Apply a rotation of (0, -60)

Tip: use [scatter3d](https://matplotlib.org/3.1.1/gallery/mplot3d/scatter3d.html) to create 3D scatter plots.

In [ ]:
#Como en el tutorial y en los ejercicios anteriores
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')

#A diferencia del ejercicio 10, ahora usamso el modelo KMeans para dibujar
kmeans = KMeans(n_clusters=3, init='random',n_init=10,tol=1e-04,random_state=0).fit(training_dt)
labels = kmeans.predict(training_dt)

# Data for a three-dimensional line
X=training_dt.loc[:,'Voice_usage'].values
Y=training_dt.loc[:,'Data_usage'].values
Z=training_dt.loc[:,'Monthly_expense'].values

ax.set_xlabel('Voice_usage')
ax.set_ylabel('Data_usage')
ax.set_zlabel('Monthly_expense')
ax.scatter3D(X, Y, Z, c=labels, cmap='viridis', linewidth=5)
ax.view_init(0, -60)
plt.show()

## Measuring the quality of the clustering and the optimal K: Elbow method and sihouette

The number of clusters to choose may not always be so obvious in real-world applications, especially if we are working with a higher dimensional dataset that cannot be visualized.

[**EX17**] [**REPORT**] Repeat the clustering (using Sklearn's or your own Kmeans function) with **K=5** and plot `Data_usage`vs `Voice_usage`scatter visualization. Can the new 5 clusters be visually distinguished? From a visual perspective, is this new cluster better than with **K=3**?

In [ ]:
kmeans = KMeans(n_clusters=5, init='random',n_init=10,tol=1e-04,random_state=0).fit(training_dt)

labels=kmeans.predict(training_dt)
voice = training_dt.loc[:,'Voice_usage']
data = training_dt.loc[:,'Data_usage']

fig = plt.figure()
plt.scatter(voice,data,c=labels)
plt.xlabel('Voice_usage')
plt.ylabel('Data_usage')
plt.show()

The elbow method is a useful graphical tool to estimate the optimal number of clusters. Intuitively, we can say that, if k increases, the distorsion within each cluster will decrease because the samples will be closer to their centroids. However, sometimes is not efficient to increase the **K** value because the distorsion doesn't decrease enough in comparision with the computation load required for higher **K**. 

Let's calculate the Elbow method for the previous dataset, i.e. containing only `Data_usage` and `Voice_usage` variables.
We use [km.inertia_](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) from the Sklearn library's KMeans to measure the sum of squared distances of samples to their closest cluster center.

In [ ]:
#Aqui usamos exactamente lo que nos daban por defecto, pero depende que compañero lo ejecutemos
#nos salta un aviso raro, aun asi sigue funcionando, quizas sea por version del anaconda

#Error
#UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2

# Selection of the dataset
training_dt=customer_dt_summary[['Voice_usage', 'Data_usage', 'Monthly_expense']]
inertia = []
#Calculate the Kmeans from K=1 to 10
for i in range(1, 11):
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=10,
        tol=1e-04, random_state=0
    )
    km.fit(training_dt)
    inertia.append(km.inertia_)

# plot
plt.plot(range(1, 11), inertia, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

[**EX18**] [**REPORT**] Repeat the Elbow method for a training dataset formed by `Voice_usage`and `Data_usage`only. Which is the optimal **K** value?

In [ ]:
# Selection of the dataset
training_dt=customer_dt_summary[['Voice_usage', 'Data_usage']]
inertia = []
#Calculate the Kmeans from K=1 to 10
for i in range(1, 11):
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=10,
        tol=1e-04, random_state=0
    )
    km.fit(training_dt)
    inertia.append(km.inertia_)

# plot
plt.plot(range(1, 11), inertia, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

**Silhouette** is a metric to measure the *quality* of the clustering process. Clustering models with a high **Silhouette** are said to be dense, i.e. samples in the same cluster are similar to each other, and well separated, where samples in different clusters are not very similar to each other. This measure has a range of [-1, 1].

[**EX19**] [**CODE**][**REPORT**]Calculate the `silhouette_score`value for a range of KMeans clusters from 2 to 7. The dataset to use is `training_dt`with the following variables: `Voice_usage`, `Data_usage` and `Monthly_expense`. Which is the value of **K** with better **Silhouette**?

Tip: use [silhouette_score](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html) to calculate the silhouette score and further information.

For a visual understanding about each cluster, we can plot the silhouette score for each sample of the dataset. Execute the following code:

In [ ]:
# Selection of the dataset
training_dt=customer_dt_summary[['Voice_usage', 'Data_usage', 'Monthly_expense']]
for j in range(2, 8):
    n_clusters=j
    km  = KMeans(j, random_state=10)
    cluster_labels = km.fit_predict(training_dt)
    silhouette_avg = silhouette_score(training_dt, cluster_labels)
    
    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(training_dt, cluster_labels)
    # Create a subplot with 1 row and 1 columns
    fig, (ax1) = plt.subplots(1,1)
    fig.set_size_inches(8, 5)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(training_dt) + (n_clusters + 1) * 10])
    
    y_lower = 10
    for i in range(j):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
 
    ax1.set_title("The silhouette plot for the various clusters")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
plt.show()

# Step 4: Insights generation: Understanding the clustering output

Thanks to **Elbow method** and the **Silhouette** metric, we can confirm that K-Means with **K=3** is good enough to cluster our customer base and generate insights for our marketing department.

[**EX20**][**REPORT**]Repeat the K-Means clustering with **K=3** for the `training_dt`formed by `CNT_EMPLOYEE`, `Turnover`, `Voice_usage`, `Data_usage`, `Monthly_expense`. For each cluster, calculate the **mean**, **standard deviation**, **min**, **max** for each variable.
- Which is the cluster with the highest voice usage? 
- Which is the cluster with the highest data usage?
- Customers in the cluster with bigger companies (i.e. bigger number or employes and turnover) use to spend more than the others customers?
- As a part of the data scientist team, which is your recommended cluster of customers to sell a new mobile tariff with unlimited data traffic? And for a new mobile tariff with unlimited voice traffic? 

In [ ]:
#Como el ejercicio 6
training_dt=customer_dt[['CNT_EMPLOYEE', 'Turnover', 'Voice_usage', 'Data_usage', 'Monthly_expense']]

kmeans = KMeans(n_clusters=3, init='random',n_init=10,tol=1e-04,random_state=0).fit(training_dt)
labels=kmeans.predict(training_dt)

print("*********** MEAN ***********")
means = training_dt.groupby(labels)[['CNT_EMPLOYEE', 'Turnover', 'Voice_usage', 'Data_usage', 'Monthly_expense']].mean()
print(means)

print("*********** STD ***********")
std = training_dt.groupby(labels)[['CNT_EMPLOYEE', 'Turnover', 'Voice_usage', 'Data_usage', 'Monthly_expense']].std()
print(std)

print("*********** MIN ***********")
mins = training_dt.groupby(labels)[['CNT_EMPLOYEE', 'Turnover', 'Voice_usage', 'Data_usage', 'Monthly_expense']].min()
print(mins)

print("*********** MAX ***********")
maxs = training_dt.groupby(labels)[['CNT_EMPLOYEE', 'Turnover', 'Voice_usage', 'Data_usage', 'Monthly_expense']].max()
print(maxs)

# Let's repeat step 3 and 4 with Mixture of Gaussians clustering

As we know, there are other mechanisms to cluster a dataset. Let's test how Mixture of Gaussians function from sklearn library works.

[**EX21**][**CODE**][**REPORT**] Execute the Mixture of Gaussians function (with number of components=3) to `training_dt` dataset with `Voice_usage`, `Data_usage`and `Monthly_expense` variables. 
- Which is the size of each cluster? 
- Visualize the scatter plot between `Data_usage` vs `Voice_usage`. Is it similar to the resulting from K-Means and K=3?
- Visualize the scatter plot between `Data_usage` vs `Monthly_expense`. Is it similar to the resulting from K-Means and K=3?
- Visualize the scatter plot between `Voice_usage` vs `Monthly_expense`. Is it similar to the resulting from K-Means and K=3?

Tip: You may use [GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) from Sklearn libray.

In [ ]:
training_dt=customer_dt[['Voice_usage', 'Data_usage', 'Monthly_expense']]

#Usamos esta vez el modelo de Mixtura de Gaussiana
gm = GaussianMixture(n_components=3, init_params="kmeans", covariance_type="full")
gm.fit(training_dt)
labels=gm.predict(training_dt)

voice = training_dt.loc[:,'Voice_usage']
data = training_dt.loc[:,'Data_usage']
month_expense = training_dt.loc[:,'Monthly_expense']

#Data_usage vs Voice_usage
fig = plt.figure()
plt.scatter(voice,data,c=labels,cmap='viridis')
plt.xlabel('Voice_usage')
plt.ylabel('Data_usage')
plt.show()

#Data_usage vs Monthly_expense
fig = plt.figure()
plt.scatter(voice,month_expense,c=labels,cmap='viridis')
plt.xlabel('Voice_usage')
plt.ylabel('Monthly_expense')
plt.show()

#Voice_usage vs Monthly_expense
fig = plt.figure()
plt.scatter(data,month_expense,c=labels,cmap='viridis')
plt.xlabel('Data_usage')
plt.ylabel('Monthly_expense')
plt.show()

[**EX22**][**REPORT**] Visualize the 3D plot  between `Voice_usage`, `Data_usage`and `Monthly_expense` after the clustering process. Apply a rotation of (0, -60)

Tip: use [scatter3d](https://matplotlib.org/3.1.1/gallery/mplot3d/scatter3d.html) to create 3D scatter plots.

In [ ]:
#De nuevo es una copia de ejercicios anteriores
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
#Ahora usamos el modelo de Mixtura de Gaussiana
gm = GaussianMixture(n_components=3, init_params="kmeans", covariance_type="full").fit(training_dt)
labels = gm.predict(training_dt)

# Data for a three-dimensional line
X=training_dt.loc[:,'Voice_usage'].values
Y=training_dt.loc[:,'Data_usage'].values
Z=training_dt.loc[:,'Monthly_expense'].values

ax.set_xlabel('Voice_usage')
ax.set_ylabel('Data_usage')
ax.set_zlabel('Monthly_expense')
ax.scatter3D(X, Y, Z, c=labels, cmap='viridis', linewidth=5)
ax.view_init(0, -60)
plt.show()

[**EX23**][**REPORT**] Evaluate the **Silhouette** metric for MoG with **number of components** from 2 to 7. Which is the number of cluster with the highest score?

In [ ]:
#Igual que el ejercicio 19 de Silhouette, pero cambiamos el modelo
# Selection of the dataset
training_dt=customer_dt_summary[['Voice_usage', 'Data_usage', 'Monthly_expense']]
for j in range(2, 8):
    n_clusters=j
    #Esta es la unica diferencia, usamos el modelo de mixtura de Gaussiana
    gm  = GaussianMixture(n_components=n_clusters, init_params="kmeans", covariance_type="full").fit(training_dt)
    cluster_labels = gm.predict(training_dt)
    silhouette_avg = silhouette_score(training_dt, cluster_labels)
    
    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(training_dt, cluster_labels)
    # Create a subplot with 1 row and 1 columns
    fig, (ax1) = plt.subplots(1,1)
    fig.set_size_inches(8, 5)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(training_dt) + (n_clusters + 1) * 10])
    
    y_lower = 10
    for i in range(j):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
 
    ax1.set_title("The silhouette plot for the various clusters")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
plt.show()

[**EX24**][**REPORT**] For n_components=3 and a dataset formed by `CNT_EMPLOYEE`, `Turnover`, `Voice_usage`, `Data_usage` and `Monthly_expense` variables, calculate the MoG clustering. For each cluster, calculate the **mean**, **standard deviation**, **min**, **max** for each variable.
- Are these clusters similar to the resulting from KMeans=3?
- Which is the cluster with the highest voice usage? 
- Which is the cluster with the highest data usage?
- Customers in the cluster with bigger companies (i.e. bigger number or employes and turnover) use to spend more than the others customers?
- As a part of the data scientist team, which is your recommended cluster of customers to sell a new mobile tariff with unlimited data traffic? And for a new mobile tariff with unlimited voice traffic? 

In [ ]:
#Copia del ejercicio 6 y 20
training_dt=customer_dt[['CNT_EMPLOYEE', 'Turnover', 'Voice_usage', 'Data_usage', 'Monthly_expense']]

gm = GaussianMixture(n_components=3, init_params="kmeans", covariance_type="full").fit(training_dt)
labels=gm.predict(training_dt)

print("*********** MEAN ***********")
means = training_dt.groupby(labels)[['CNT_EMPLOYEE', 'Turnover', 'Voice_usage', 'Data_usage', 'Monthly_expense']].mean()
print(means)

print("*********** STD ***********")
std = training_dt.groupby(labels)[['CNT_EMPLOYEE', 'Turnover', 'Voice_usage', 'Data_usage', 'Monthly_expense']].std()
print(std)

print("*********** MIN ***********")
mins = training_dt.groupby(labels)[['CNT_EMPLOYEE', 'Turnover', 'Voice_usage', 'Data_usage', 'Monthly_expense']].min()
print(mins)

print("*********** MAX ***********")
maxs = training_dt.groupby(labels)[['CNT_EMPLOYEE', 'Turnover', 'Voice_usage', 'Data_usage', 'Monthly_expense']].max()
print(maxs)


[**EX25**][**REPORT**] In this project we have used several clustering techniques to segment a customer base with aim to have a clear understanding about how they are. It is a very useful application in marketing to identify clusters or groups of target customers to sell a product or to focus a marketing campaign. 
However, clustering have a lot of applications. In this exercise we will apply to other use case:

We have clustered the population in an area of Barcelona according to several criterias:

**Clustering execution 1:** is based on `number_family_members`, `electricity_consumption`and `water_consumption`. The output is the following:

![<PJ1_clustering_1_plot>](PJ1_clustering_1.png)

**Clustering execution 2:** is based on `number_pets`, `size_home`and `number_kids`. The output is the following:
![<PJ1_clustering_2_plot>](PJ1_clustering_2.png)

**Clustering execution 3:** is based on `education_level`, `salary`and `number_vehicles`. The output is the following:
![<PJ1_clustering_3_plot>](PJ1_clustering_3.png)

The caracteristic (average+deviation) of each variable that describes each segment (or cluster) for each clustering process is the following:
![<clustering_summary_plot>](clustering_summary.png)



Answer the following questions considering the previous maps and the description of each cluster (table above):
- Describe the homes located in the map:
![<homes_map_plot>](homes_map.png)
- Which clustering execution has produced more homogeneous groups? Justify your answer
- Which clustering execution should have better silhoette? Justify your anwer
- If we would like to build a **fast food** restaurant, where would you place it? Justify your answer
- If we would like to place a **school**, where would you place it? Justify your answer
- In case we would like to place a **laundry**, which 3 variables you would select from the following dataset to launch a new clustering process:
`number_family_members`, `monthly_dress_expense`, `number_shoes`, `electricity_consumption`,`number_cars`,`monthly_mobile_expense`, `number_rooms`, `water_consumption`, `number_pets`, `size_home`, `number_kids`, `education_level`, `salary` and `number_vehicles`.


# Deliver

Deliver:

* A zip file containing your notebook (.ipynb file) with all the [**CODE**] parts implemented.
* A 5-pages PDF report including all parts of this notebook marked with "[**REPORT**]"

The report should end with the following statement: **We hereby declare that, except for the code provided by the course instructors, all of our code, report, and figures were produced by ourselves.**